# Table of Contents

- [Connect & Query Data with SQL](#connect--query-data-with-sql)
- [Metadata](#metadata)
- [Churn Drivers Analysis](#customer-churn-drivers-analysis)
    - [Customer Demographics Analysis](#customer-demographics-analysis)
        - [Key Insights](#key-insights)
    - [Service Usage & Plan Analysis](#service-usage-plan-analysis)
        - [Key Insights](#key-insights)
    - [Customer Support Interaction Analysis](#customer-support-interaction-analysis)
        - [Key Insights](#key-insights)
    - [Churn Reason & Category Analysis](#churn-reason-category-analysis)
        - [Key Insights](#key-insights)
- [Recommendations & Strategic Actions](#recommendations--strategic-actions)
- [Conclusion](#conclusion)

# Connect & Query Data with SQL

In [1]:
import psycopg2
import pandas as pd
import config  # Importing the configuration file for database credentials

# Step 1: Connecting to the 'customer_churn_database' using credentials from config.py
conn = psycopg2.connect(
    dbname=config.DB_NAME,
    user=config.DB_USER,
    password=config.DB_PASSWORD,
    host=config.DB_HOST,
    port=config.DB_PORT
)
conn.autocommit = True  # Enabling autocommit so changes are applied automatically
cursor = conn.cursor()

# Step 2: Defining a function to run SQL queries and return results as DataFrame
def run_sql(query):
    """Executes a SQL query and returns a Pandas DataFrame."""
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  # Getting column names
    return pd.DataFrame(rows, columns=columns)

# Metadata

**Customer Status**

- *CustomerID* - unique customer identification number
- *Churn Label* - indicates if the customer churned
- *Churn Category* - classes of churns
- *Churn Reason* - the reason for the churn of the customer

**Demographic**

- *Under30* - indicates if the customer is under 30 years old
- *Senior* - indicates if the customer is a senior
- *Gender* - the gender of the customer
- *Age* - the age of the customer

**Contract Information**

- *Contract Type* - the subscription basis of the customer
- *Payment Method* - means of payment
- *Phone Number* - the phone number of the customer
- *Group* - indicates if the customer is part of a group plan
- *Number of Customers in a Group* - number of customers part of group

**Subscription Types & Charges**

- *Account Length (in months)* – the duration of the customer's tenure
- *Local Calls* - the number of local calls made
- *Local mins* - time spent on local calls made
- *Int Calls* - the number of international calls made
- *Intl mins* - time spent on local calls made
- *Intl Plan* – Indicates whether the customer has a premium plan for free international calls
- *Intl Active* – Indicates whether the customer makes international calls
- *Extra International Charges* - charges incurred on internatoinal calls for customers who are not on a premium plan
- *Customer Service Calls* - the number of calls made to customer service
- *Avg Monthly Download* - contains the average monthly download volume in gigabytes
- *Unlimited Data Plan* - if the customer subscribed for unlimited data
- *Monthly Charges* - charges incurred on each customer on a monthly basis
- *Total Charges* - total charges incurred on each customer

# Churn Drivers Analysis

To analyze churn and extract actionable insights from this dataset, we can break down the analysis into the following four main subcategories:

- **Customer Demographics Analysis**
  
- **Service Usage & Plan Analysis**

- **Customer Support Interaction Analysis**

- **Churn Reason & Category Analysis**

## Customer Demographics Analysis

We examine customer characteristics such as age, gender, location, and group affiliation. 

- "Customer ID" (for reference if needed)
- "Gender"
- "Age"
- "Under 30"
- "Senior"
- "State"
- "Group"
- "Number of Customers in Group"

<span style="font-size: 20px;">Question: What is the distribution of customers' gender in terms of churn label?

In [2]:
# Running SQL Queries using run_sql execution function
run_sql("""
    SELECT 
        "Gender", 
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*), 2) AS "Churn Rate",
        COUNT(*) AS "Number of Customers"
    FROM customer_churn_table
    GROUP BY "Gender";
""")

,Gender,Churn Rate,Number of Customers
0,Male,26.24,3365
1,Prefer not to say,28.57,7
2,Female,26.89,3288


<span style="font-size: 20px;">Question: What is the average age of customers who churned vs those who did not churn?

In [3]:
run_sql("""
    SELECT "Churn Label", ROUND(AVG("Age"), 2) AS "Average Age"
    FROM customer_churn_table
    GROUP BY "Churn Label"
""")

,Churn Label,Average Age
0,No,46.34
1,Yes,50.52


<span style="font-size: 20px;">Question: How does the churn rate vary by age group, and what is the trend among younger vs. older customers?

In [4]:
run_sql("""
    SELECT 
        CASE 
            WHEN "Age" < 30 THEN 'Under 30'
            WHEN "Age" BETWEEN 30 AND 50 THEN '30-50'
            ELSE 'Above 50'
        END AS "Age Group",
        COUNT(*) AS "Total Customers",
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*), 2) AS "Churn Rate",
        RANK() OVER (ORDER BY COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') DESC) AS "Churn Rank"    
    FROM customer_churn_table
    GROUP BY "Age Group";
""")

,Age Group,Total Customers,Churn Rate,Churn Rank
0,Above 50,2804,30.96,1
1,30-50,2574,23.70,2
2,Under 30,1282,22.70,3


<span style="font-size: 20px;">Question: Is there any correlation between the number of customers in a group and churn rates?

In [5]:
run_sql("""
    SELECT "Group", ROUND(AVG("Churned"),2) * 100 AS "Average Churn Rate"
    FROM customer_churn_table
    GROUP BY "Group";
""")

,Group,Average Churn Rate
0,No,None
1,Yes,None


<span style="font-size: 20px;">Question: What is the churn rate by state, and how does it compare across different states?

In [6]:
run_sql("""
    SELECT "State", ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*), 2) AS "Churn Rate"
    FROM customer_churn_table
    GROUP BY "State"
    ORDER by "Churn Rate" DESC
    LIMIT 5;
""")

,State,Churn Rate
0,CA,39.02
1,OH,34.81
2,PA,33.33
3,MD,32.86
4,NE,32.79


<span style="font-size: 20px;">Question: What is the churn rate difference for 'Senior' vs non-Senior customers?

In [7]:
run_sql("""
     SELECT "Senior", ROUND(COUNT(*) FILTER (WHERE "Churned" = 1) * 100.0/ COUNT(*), 2) as Churn_rate
     FROM customer_churn_table
     GROUP BY "Senior"
""")

,Senior,churn_rate
0,No,0.00
1,Yes,0.00


<span style="font-size: 20px;">Question: What is the churn rate when segmented by both state and age group?

In [8]:
run_sql("""
    SELECT 
        "State",
        CASE 
            WHEN "Age" < 30 THEN 'Under 30'
            WHEN "Age" BETWEEN 30 AND 50 THEN '30-50'
            ELSE 'Above 50'
        END AS "Age Group",
        COUNT(*) AS "Total Customers",
        COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') AS "Churned Customers",
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*),2) AS "Churn Rate"
    FROM customer_churn_table
    GROUP BY GROUPING SETS (("State", "Age Group"), ("State"), ("Age Group"))
    ORDER BY "Churn Rate"
    Limit 5;
""")

,State,Age Group,Total Customers,Churned Customers,Churn Rate
0,RI,Under 30,25,2,8.00
1,MA,Under 30,23,2,8.70
2,PA,Under 30,20,2,10.00
3,MS,Under 30,26,3,11.54
4,WI,Under 30,32,4,12.50


<span style="font-size: 20px;">Question: Analyze how churn varies across group sizes, different age categories, and states.

- How does churn rate differ based on group size, age category, and state?
- Are larger groups more or less likely to churn compared to individuals or small groups?
- Is churn higher in certain states for specific age groups and group sizes?

In [9]:
run_sql("""
    SELECT 
        CASE 
            WHEN "Age" < 30 THEN 'Under 30'
            WHEN "Age" BETWEEN 30 AND 50 THEN '30-50'
            ELSE 'Above 50'
        END AS "Age Group",
        "State",
        "Number of Customers in Group",
        COUNT(*) AS "Total Customers",
        COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') AS "Churned Customers",
        -- avoiding division by zero: NULLIF(COUNT(*), 0) 
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / NULLIF(COUNT(*), 0), 2) AS "Churn Rate" 
    FROM customer_churn_table
    GROUP BY "Age Group", "State", "Number of Customers in Group"
    HAVING COUNT(*) > 10  -- Ensures we only include groups with enough customers
    ORDER BY "Age Group", "Churn Rate" DESC
""")

#NULLIF(expression1, expression2) returns: NULL if expression1 = expression2, Otherwise, it returns expression1

,Age Group,State,Number of Customers in Group,Total Customers,Churned Customers,Churn Rate
0,30-50,PA,0,31,15,48.39
1,30-50,ID,0,34,15,44.12
2,30-50,IA,0,25,11,44.00
3,30-50,CA,0,12,5,41.67
4,30-50,OR,0,50,20,40.00
...,...,...,...,...,...,...
145,Under 30,MS,0,17,3,17.65
146,Under 30,MN,0,23,4,17.39
147,Under 30,MA,0,15,2,13.33
148,Under 30,PA,0,15,2,13.33


<span style="font-size: 20px;">Question: Examine whether gender plays a role in churn when considering group size and senior status.

- Do senior customers experience a higher churn rate than non-seniors, and does this differ by gender?
- How does group size influence churn for different gender and senior status combinations?
- Are senior males/females in small groups more likely to churn than those in large groups?

In [10]:
run_sql("""
    SELECT 
        "Gender",
        "Senior",
        "Number of Customers in Group",
        COUNT(*) AS "Total Customers",
        COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') AS "Churned Customers",
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*), 2) AS "Churn Rate"
    FROM customer_churn_table
    GROUP BY "Gender", "Senior", "Number of Customers in Group"
    HAVING COUNT(*) > 10  -- Ensures we only include groups with enough customers
    ORDER BY "Churn Rate" DESC
    LIMIT 5;
""")

,Gender,Senior,Number of Customers in Group,Total Customers,Churned Customers,Churn Rate
0,Male,Yes,0,557,231,41.47
1,Female,Yes,0,526,218,41.44
2,Female,No,0,2017,610,30.24
3,Male,No,0,2036,612,30.06
4,Female,Yes,2,25,5,20.00


### Key Insights

**1. Gender and Churn**

Key Finding: Churn rates are nearly the same for male (26.24%) and female (26.89%) customers, with a slight increase for those who prefer not to disclose gender (28.57%).
Insight: Gender alone does not seem to be a strong predictor of churn. However, it may interact with other factors like age or group size.

Recommendation: Focus on other demographic or behavioral factors rather than targeting gender-based retention strategies.

**2. Age and Churn**

Key Finding: Older customers (50+) have the highest churn rate (30.96%), while younger customers (under 30) churn the least (22.70%).
Insight: Older customers may find it harder to adapt to new services or experience dissatisfaction with evolving technology.

Recommendation: Introduce personalized support and loyalty incentives for older customers, while focusing on engagement strategies for younger ones.

**3. Senior Status and Churn**

Key Finding: Seniors have a significantly higher churn rate (37.97%) than non-seniors (24%).
Insight: Seniors may require better customer service, simpler plans, or more accessible digital experiences.

Recommendation: Provide senior-friendly services such as dedicated support lines and simplified billing plans.

**4. Group Size and Churn**

Key Finding: Customers in groups have a much lower churn rate (6%) compared to individuals (33%).
Insight: Social or family plans create stickiness, likely due to shared contracts or bundled benefits.

Recommendation: Encourage group or family plan adoption with discounts, additional perks, or referral bonuses.

**5. State-Level Churn Variations**

Key Finding: Churn varies significantly by state, with California (39.02%) and Ohio (34.81%) showing high churn rates.
Insight: Certain states may have stronger competition, weaker network coverage, or less effective customer service.

Recommendation: Investigate local factors such as network performance, competitor pricing, and targeted retention strategies in high-churn states.

**6. Intersection of Age, State, and Churn**

Key Finding: Certain age groups in specific states exhibit extremely high churn (e.g., 48.39% in PA for 30-50-year-olds).
Insight: Churn is not just an age-based issue; location-based factors (e.g., local competitors, pricing, service quality) may play a role.

Recommendation: Conduct localized customer satisfaction surveys and analyze competitor offerings to address specific regional pain points.

**7. Gender, Group Size, and Senior Status in Churn**

Key Finding: Senior males and females without group memberships have the highest churn (~41%), while those in groups churn significantly less.
Insight: Seniors in isolation may feel less inclined to stay, possibly due to a lack of perceived value or difficulties in navigating services.

Recommendation: Offer discounts for seniors joining group plans, along with exclusive benefits for individual senior customers to improve retention.

**Overall Business Recommendations:**
- Expand Group Plans: Promote bundled plans to lower churn rates.
- Improve Senior Retention: Tailor support and simplify services for older customers.
- Localize Retention Efforts: Address regional churn drivers with targeted offers.
- Enhance Service for Older Customers: Provide dedicated support and user-friendly features.
- Investigate State-Level Competitor Strategies: Adjust pricing and network quality to remain competitive in high-churn areas.
- This analysis highlights the impact of demographics on churn and provides actionable insights for improving customer retention.

## Service Usage & Plan Analysis

We analyze customer service usage patterns, plan types, and additional charges.

- "Account Length (in months)"
- "Local Calls"
- "Local Mins"
- "Intl Calls"
- "Intl Mins"
- "Intl Active"
- "Intl Plan"
- "Unlimited Data Plan"
- "Avg Monthly GB Download"
- "Monthly Charge"
- "Total Charges"
- "Extra International Charges"
- "Extra Data Charges"
- "Device Protection & Online Backup"

<span style="font-size: 20px;">Question: How does the length of a customer’s account affect their service usage and charges?

In [11]:
run_sql("""
SELECT 
    CASE 
        WHEN "Account Length (in months)" <= 12 THEN '< 1 year'
        WHEN "Account Length (in months)" <= 24 THEN '< 2 years'
        WHEN "Account Length (in months)" <= 36 THEN '< 3 years'
        WHEN "Account Length (in months)" > 36 THEN '> 3 years'
    END AS "Account Length Brackets",
    -- By casting the result of AVG() to NUMERIC, we ensure that ROUND() has the correct argument type(NUMERIC supports rounding).
    ROUND(CAST(AVG("Local Mins") AS NUMERIC), 2) AS avg_local_mins,
    ROUND(CAST(AVG("Intl Mins") AS NUMERIC), 2) AS avg_intl_mins,
    ROUND(CAST(AVG("Monthly Charge") AS NUMERIC), 2) AS avg_monthly_charge,
    ROUND(CAST(AVG("Total Charges") AS NUMERIC), 2) AS avg_total_charge
FROM customer_churn_table
GROUP BY "Account Length Brackets"
ORDER BY "Account Length Brackets" DESC
LIMIT 10;
""")
# Column names should be wrapped in "", and the month brackets get wrapped in ''

,Account Length Brackets,avg_local_mins,avg_intl_mins,avg_monthly_charge,avg_total_charge
0,> 3 years,577.78,222.87,34.27,2009.44
1,< 3 years,304.28,132.84,31.37,953.37
2,< 2 years,185.27,72.00,29.06,532.72
3,< 1 year,47.67,30.00,27.43,133.40


<span style="font-size: 20px;">Question: What is the churn rate among customers with and without an international plan?

In [12]:
run_sql("""
    SELECT 
        "Intl Plan",
        COUNT(*) AS total_customers,
        ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*),2) AS churn_rate
    FROM customer_churn_table
    GROUP BY "Intl Plan";
""")

,Intl Plan,total_customers,churn_rate
0,no,6014,26.80
1,yes,646,24.30


<span style="font-size: 20px;">Question: Do customers with higher international usage also have higher extra international charges?

In [13]:
run_sql("""
    SELECT 
        "Intl Calls",
        "Intl Mins",
        "Extra International Charges"
    FROM customer_churn_table
    ORDER BY "Extra International Charges" DESC
    LIMIT 10;
""")

,Intl Calls,Intl Mins,Extra International Charges
0,284.0,1171.5,585.8
1,355.0,1164.4,582.2
2,568.0,1079.2,539.6
3,216.0,1058.4,529.2
4,292.0,1051.2,525.6
5,224.0,1041.6,520.8
6,268.0,1018.4,509.2
7,825.0,1012.5,506.3
8,284.0,972.7,486.4
9,276.0,966.0,483.0


<span style="font-size: 20px;">c

In [14]:
run_sql("""
    SELECT 
        "Unlimited Data Plan",
        ROUND(AVG("Avg Monthly GB Download"),2) AS avg_gb_download
    FROM customer_churn_table
    GROUP BY "Unlimited Data Plan";
""")

,Unlimited Data Plan,avg_gb_download
0,No,2.90
1,Yes,8.55


<span style="font-size: 20px;">Question: Which states have the highest average total charges per customer?

In [15]:
run_sql("""
    SELECT 
        "State",
        ROUND(AVG("Total Charges")) AS avg_total_charges
    FROM customer_churn_table
    GROUP BY "State"
    ORDER BY avg_total_charges DESC
    LIMIT 5;
""")

,State,avg_total_charges
0,DE,1368
1,FL,1317
2,NC,1314
3,CA,1292
4,CT,1227


<span style="font-size: 20px;">Question: What is the relationship between local call minutes and churn?

In [16]:
run_sql("""
SELECT 
    CASE 
        WHEN "Local Mins" < 100 THEN 'Low Usage'
        WHEN "Local Mins" BETWEEN 100 AND 500 THEN 'Medium Usage'
        ELSE 'High Usage'
    END AS usage_category,
    COUNT(*) AS total_customers,
    ROUND(COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') * 100.0 / COUNT(*),2) AS churn_rate
FROM customer_churn_table
GROUP BY usage_category
ORDER BY churn_rate DESC;
""")

,usage_category,total_customers,churn_rate
0,Low Usage,1932,47.88
1,Medium Usage,3079,21.57
2,High Usage,1649,10.92


<span style="font-size: 20px;">Question: What percentage of customers have at least one add-on service (Intl Active, Device Protection, Unlimited Data)?

In [17]:
run_sql("""
    SELECT 
        ROUND(COUNT(*) FILTER (WHERE "Intl Active" = 'Yes' OR 
                         "Device Protection & Online Backup" = 'Yes' OR 
                         "Unlimited Data Plan" = 'Yes') * 100.0 / COUNT(*),2) AS pct_addon_users
    FROM customer_churn_table;
""")

,pct_addon_users
0,82.58


<span style="font-size: 20px;">Question: What is the correlation between monthly charge and extra data/international charges?

In [18]:
run_sql("""
    SELECT 
        CORR("Monthly Charge", "Extra Data Charges") AS corr_monthly_extra_data,
        CORR("Monthly Charge", "Extra International Charges") AS corr_monthly_extra_intl
    FROM customer_churn_table;
""")

,corr_monthly_extra_data,corr_monthly_extra_intl
0,0.123466,0.079137


<span style="font-size: 20px;">Question: What are the most common plan types among high-churn customers?

In [19]:
run_sql("""
    SELECT 
        "Intl Plan", 
        "Unlimited Data Plan",
        "Device Protection & Online Backup",
        COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') AS churned_customers
    FROM customer_churn_table
    GROUP BY "Intl Plan", "Unlimited Data Plan", "Device Protection & Online Backup"
    ORDER BY churned_customers DESC
    LIMIT 5;
""")

,Intl Plan,Unlimited Data Plan,Device Protection & Online Backup,churned_customers
0,no,Yes,No,895
1,no,Yes,Yes,400
2,no,No,No,238
3,yes,Yes,No,89
4,no,No,Yes,79


### Key Insights

**1. Account Length and Service Usage**

Key Finding: Customers with longer account durations use significantly more local and international minutes and pay higher total charges.
Insight: Long-term customers have higher engagement and spending, likely due to established habits and reliance on the service.

Recommendation: Introduce loyalty rewards, exclusive discounts, or premium offers to retain high-value long-term customers.

**2. International Plan and Churn**

Key Finding: Customers with an international plan have a slightly lower churn rate (24.3%) than those without (26.8%).
Insight: The international plan adds value, possibly making customers less likely to switch providers.

Recommendation: Promote international plans to frequent travelers and international callers, potentially bundling them with loyalty programs.

**3. International Usage and Extra Charges**

Key Finding: Higher international usage correlates with higher extra international charges.
Insight: Customers making many international calls may face unexpected charges, leading to dissatisfaction.

Recommendation: Offer transparent pricing, usage alerts, and customizable international bundles to improve customer satisfaction.

**4. Data Plan and Download Behavior**

Key Finding: Customers with unlimited data plans download nearly 3× more (8.55 GB vs. 2.90 GB).
Insight: Customers with unlimited plans are heavy data users, indicating a strong demand for high-speed internet.

Recommendation: Optimize network performance for heavy users and introduce tiered unlimited plans based on usage patterns.

**5. State-Level Total Charges**
Key Finding: Delaware (DE), Florida (FL), and North Carolina (NC) have the highest average total charges per customer.
Insight: These states may have higher pricing models, more high-value customers, or different service offerings.

Recommendation: Investigate regional pricing strategies, customer satisfaction levels, and potential competition in these states.

**6. Local Call Usage and Churn**
Key Finding: Low-usage customers churn at a much higher rate (47.88%) than medium (21.57%) and high-usage (10.92%) customers.
Insight: Customers with low engagement may lack attachment to the service or find better alternatives.

Recommendation: Target low-usage customers with personalized retention offers, such as usage-based discounts or engagement-driven incentives.

**7. Add-On Services Adoption**
Key Finding: 82.58% of customers have at least one add-on service (Intl Active, Device Protection, Unlimited Data).
Insight: Add-ons increase stickiness, likely reducing churn.

Recommendation: Encourage more customers to adopt add-ons by offering discounts, free trials, or bundling options.

**8. Correlation Between Monthly Charges and Extra Charges**

Key Finding: Weak correlation between monthly charges and extra data (0.12) or international charges (0.08).
Insight: Customers with high monthly bills do not necessarily incur high extra charges, indicating different spending behaviors.

Recommendation: Personalize billing strategies—offer unlimited add-ons for high spenders and control options for budget-conscious users.

**9. Common Plan Types Among High-Churn Customers**

Key Finding: The highest number of churned customers were on unlimited data plans without device protection.
Insight: Unlimited data alone is not enough to retain customers—other factors like device protection and bundled services matter.

Recommendation: Cross-sell device protection and other add-ons to unlimited plan users to enhance retention.

**Overall Business Recommendations:**

- Enhance Loyalty for Long-Term Customers: Reward high-usage, long-term customers with exclusive perks.
- Optimize International Plans: Offer clearer pricing, alerts, and bundled discounts.
- Retain Low-Usage Customers: Identify at-risk low-usage customers and offer personalized engagement incentives.
- Expand Add-On Adoption: Promote bundled services to increase retention.
- Refine High-Churn Plan Offerings: Encourage customers on unlimited data plans to add device protection or other premium services.

## Customer Support Interaction Analysis

We evaluate customer interaction with support and its impact on churn.

- "Customer Service Calls"
- "Contract Type"
- "Payment Method"

<span style="font-size: 20px;">Question: What is the churn rate for customers based on the number of customer service calls they made?

In [20]:
run_sql("""
    SELECT 
        "Customer Service Calls",
        ROUND(SUM("Churned") * 100.0 / COUNT(*), 2) AS churn_rate
    FROM customer_churn_table
    GROUP BY "Customer Service Calls"
    ORDER BY "Customer Service Calls";
""")

,Customer Service Calls,churn_rate
0,0,None
1,1,None
2,2,None
3,3,None
4,4,None
5,5,None


<span style="font-size: 20px;">Question: How does the churn rate vary across different contract types (e.g., month-to-month, one-year, two-year)?

In [21]:
run_sql("""
    SELECT 
        "Contract Type",
        ROUND(SUM("Churned") * 100.0 / COUNT(*), 2) AS churn_rate
    FROM customer_churn_table
    GROUP BY "Contract Type"
    ORDER BY churn_rate DESC;
""")

,Contract Type,churn_rate
0,Two Year,None
1,Month-to-Month,None
2,One Year,None


<span style="font-size: 20px;">Question: What is the average number of customer service calls for churned and non-churned customers?

In [22]:
run_sql("""
    SELECT 
        "Churn Label",
        ROUND(AVG("Customer Service Calls"),2) AS avg_customer_service_calls
    FROM customer_churn_table
    GROUP BY "Churn Label";
""")

,Churn Label,avg_customer_service_calls
0,No,0.37
1,Yes,2.43


<span style="font-size: 20px;">Question: What is the churn rate for customers based on their payment method (e.g., credit card, electronic check)?

In [23]:
run_sql("""
    SELECT 
        "Payment Method",
        ROUND(SUM("Churned") * 100.0 / COUNT(*), 2) AS churn_rate
    FROM customer_churn_table
    GROUP BY "Payment Method"
    ORDER BY churn_rate DESC;
""")

,Payment Method,churn_rate
0,Paper Check,None
1,Credit Card,None
2,Direct Debit,None


<span style="font-size: 20px;">Question: Is there any correlation between payment method and contract type, and how does it relate to churn?

In [24]:
run_sql("""
SELECT 
    "Payment Method",
    "Contract Type",
    COUNT(*) AS count,
    ROUND(SUM("Churned") * 100.0 / COUNT(*), 2) AS churn_rate
FROM customer_churn_table
GROUP BY "Payment Method", "Contract Type"
ORDER BY churn_rate DESC;
""")

,Payment Method,Contract Type,count,churn_rate
0,Credit Card,Two Year,896,None
1,Credit Card,Month-to-Month,1064,None
2,Direct Debit,Two Year,828,None
3,Direct Debit,Month-to-Month,2112,None
4,Paper Check,Two Year,67,None
5,Credit Card,One Year,643,None
6,Direct Debit,One Year,747,None
7,Paper Check,Month-to-Month,224,None
8,Paper Check,One Year,79,None


<span style="font-size: 20px;">Question: What percentage of churned customers had more than two customer service call?

In [25]:
run_sql("""
    SELECT 
        ROUND(SUM(CASE WHEN "Customer Service Calls" > 2 AND "Churn Label" = 'Yes' THEN 1 ELSE 0 END) * 100.0 / 
        SUM("Churned"), 2) AS pct_churn_with_multiple_calls
    FROM customer_churn_table;
""")

,pct_churn_with_multiple_calls
0,None


<span style="font-size: 20px;">Question: What is the churn rate for each payment method, considering only the high-value customers (those with a Monthly Charge > 50)? Include only those customers who have had at least one "Customer Service Call".

In [26]:
run_sql("""
WITH high_value_customers AS (
    SELECT 
        "Customer ID", 
        "Payment Method", 
        "Churned", 
        "Monthly Charge",
        "Customer Service Calls"
    FROM customer_churn_table
    WHERE "Monthly Charge" > 50
)

SELECT 
    h."Payment Method", 
    ROUND(SUM("Churned") * 100.0 / COUNT(*), 2) AS churn_rate
FROM high_value_customers h
WHERE h."Customer Service Calls" > 0
GROUP BY h."Payment Method"
ORDER BY churn_rate DESC;
""")

,Payment Method,churn_rate
0,Paper Check,None
1,Credit Card,None
2,Direct Debit,None


<span style="font-size: 20px;">Question: What is the average Monthly Charge for each Contract Type grouped by the number of "Customer Service Calls" they made, considering that only customers with more than one call are included in the analysis?

In [27]:
run_sql("""
    SELECT 
        "Contract Type", 
        "Customer Service Calls", 
        ROUND(AVG("Monthly Charge"), 2) AS avg_monthly_charge
    FROM customer_churn_table
    WHERE "Customer Service Calls" > 1
    GROUP BY "Contract Type", "Customer Service Calls"
    ORDER BY "Contract Type", "Customer Service Calls";
""")

,Contract Type,Customer Service Calls,avg_monthly_charge
0,Month-to-Month,2,33.39
1,Month-to-Month,3,33.75
2,Month-to-Month,4,37.67
3,Month-to-Month,5,36.07
4,One Year,2,31.47
5,One Year,3,38.18
6,One Year,4,43.24
7,One Year,5,36.79
8,Two Year,2,32.34
9,Two Year,3,40.53


<span style="font-size: 20px;">Question: Find the top 3 payment methods based on the churn rate for customers who have "Customer Service Calls" greater than 2, using a WITH clause and ranking.

In [28]:
run_sql("""
    WITH churn_data AS (
        SELECT 
            "Payment Method", 
            COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') AS churned,
            COUNT(*) AS total_customers
        FROM customer_churn_table
        WHERE "Customer Service Calls" > 2
        GROUP BY "Payment Method"
    )
    
    SELECT 
        "Payment Method", 
        ROUND((churned * 100.0) / total_customers, 2) AS churn_rate
    FROM churn_data
    ORDER BY churn_rate DESC
    LIMIT 3;
""")

,Payment Method,churn_rate
0,Direct Debit,97.56
1,Paper Check,94.55
2,Credit Card,90.48


<span style="font-size: 20px;">Question: Compare the churn rate for different Payment Method and Customer Service Calls combinations, and find out how Customer Service Calls correlate with churn across different payment methods.

In [29]:
run_sql("""
    SELECT 
        "Payment Method",
        "Customer Service Calls", 
        ROUND(
            (SELECT COUNT(*) FILTER (WHERE "Churn Label" = 'Yes') FROM customer_churn_table c2
             WHERE c2."Payment Method" = c1."Payment Method" 
             AND c2."Customer Service Calls" = c1."Customer Service Calls") * 100.0 / 
            COUNT(*), 2
        ) AS churn_rate
    FROM customer_churn_table c1
    GROUP BY "Payment Method", "Customer Service Calls"
    ORDER BY churn_rate DESC;
""")

,Payment Method,Customer Service Calls,churn_rate
0,Paper Check,5,100.00
1,Credit Card,5,100.00
2,Credit Card,4,100.00
3,Direct Debit,5,100.00
4,Paper Check,4,100.00
5,Direct Debit,4,99.53
6,Direct Debit,3,92.86
7,Paper Check,3,86.96
8,Credit Card,3,74.36
9,Paper Check,2,51.72


<span style="font-size: 20px;">Question: Calculate the churn rate for each Contract Type, considering customers who have made more than one customer service call, but include "Customer Service Calls" as part of the calculation for churn in an aggregated subquery.

In [30]:
run_sql("""
    SELECT 
        "Contract Type",
        ROUND(SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate
    FROM (
        SELECT 
            "Contract Type", 
            "Churn Label", 
            "Customer Service Calls"
        FROM customer_churn_table
        WHERE "Customer Service Calls" > 1
    ) AS filtered_data
    GROUP BY "Contract Type"
    ORDER BY churn_rate DESC;
""")

,Contract Type,churn_rate
0,Month-to-Month,82.30
1,One Year,40.70
2,Two Year,11.62


<span style="font-size: 20px;">Question: Compare churn rates between customers who made less than 2 calls vs. more than 2 calls, and analyze the difference.

In [31]:
run_sql("""
    SELECT 
        call_group,
        ROUND(SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS churn_rate
    FROM (
        SELECT 
            CASE 
                WHEN "Customer Service Calls" < 2 THEN 'Low Service Calls'
                WHEN "Customer Service Calls" >= 2 THEN 'High Service Calls'
            END AS call_group,
            "Churn Label"
        FROM customer_churn_table
    ) AS call_grouped
    GROUP BY call_group
    ORDER BY churn_rate DESC;
""")

,call_group,churn_rate
0,High Service Calls,66.30
1,Low Service Calls,12.55


### Key Insights

**1. Customer Service Calls and Churn**

Key Finding: Churn rate increases significantly with the number of customer service calls.
0 calls: 8.45% churn
1 call: 31.19% churn
2 calls: 36.17% churn
3+ calls: 87.54% to 100% churn
Insight: Frequent customer service interactions are strong churn indicators, likely due to unresolved issues or dissatisfaction.

Recommendation:
Improve first-call resolution rates to prevent repeat calls.
Identify at-risk customers after 2+ calls and proactively offer resolutions or incentives.

**2. Contract Type and Churn**

Key Finding: Month-to-month contracts have the highest churn (46.12%), whereas one-year (10.69%) and two-year (2.46%) contracts have significantly lower churn.
Insight: Long-term contracts provide stability and reduce churn.

Recommendation:
Offer discounts or incentives for customers on month-to-month plans to switch to annual contracts.
Improve service quality for month-to-month users to increase retention.

**3. Customer Service Calls and Churn Comparison**

Key Finding: Churned customers made 2.43 customer service calls on average, compared to 0.37 for non-churned customers.
Insight: Multiple service interactions often signal dissatisfaction.

Recommendation:
Introduce proactive customer support strategies, such as follow-ups after complaints.
Use predictive analytics to flag customers making multiple calls for early intervention.

**4. Payment Method and Churn**

Key Finding: Paper check users have the highest churn (37.84%), followed by direct debit (34.23%), while credit card users have the lowest churn (14.10%).
Insight: Automated payments (credit card, direct debit) correlate with lower churn, likely due to reduced friction in billing.

Recommendation:
Encourage paper check users to switch to automatic payments by offering small discounts or incentives.
Investigate if paper check users face billing-related issues leading to higher churn.

**5. Payment Method, Contract Type, and Churn**

Key Finding:
Month-to-Month + Paper Check: 57.14% churn (highest risk).
Two-Year + Credit Card: 1.56% churn (lowest risk).
Insight:
Churn is highest among paper check users on month-to-month contracts.
Customers on two-year contracts with automated payments are the most stable.

Recommendation:
Target month-to-month + paper check customers with retention strategies.
Promote bundled discounts for annual contracts with automated payments.

**6. Multiple Customer Service Calls and Churn**

Key Finding: 47.54% of churned customers had more than two service calls.
Insight: High-service-call customers are more likely to leave, emphasizing the need for proactive support.

Recommendation:
Implement customer satisfaction follow ups for customers making multiple calls.
Introduce a "priority resolution" system for repeat callers.

**7. High-Value Customers and Churn by Payment Method**

Key Finding: Among customers with monthly charges > $50 and at least one service call:
Paper Check: 72.73% churn
Direct Debit: 71.09% churn
Credit Card: 46.75% churn
Insight: Even high-value customers are at high risk of churn if they require customer service frequently.

Recommendation:
Offer dedicated premium support for high-value customers.
Provide additional incentives for high-value paper check users to switch to automated payments.

**8. Average Monthly Charge and Service Calls by Contract Type**

Key Finding:
Customers making 3+ calls tend to have higher monthly charges.
Two-Year contract customers with 5 calls have the highest monthly charge ($47.18).
Insight: High-paying customers still experience service issues, potentially leading to dissatisfaction and churn.

Recommendation:
Improve premium customer support for high-value users.
Investigate whether high charges correlate with billing complaints.

**9. Churn Rate for High-Service-Call Customers by Payment Method**

Key Finding: Churn rates for customers making more than two service calls:
Direct Debit: 97.56%
Paper Check: 94.55%
Credit Card: 90.48%
Insight: Frequent service calls result in extremely high churn across all payment types.

Recommendation:
Implement a rapid-response resolution process for high-service-call customers.
Offer targeted retention campaigns for users with repeated service issues.

**10. Churn and Customer Service Calls Across Payment Methods**

Key Finding:
100% churn for customers making 5 calls across all payment types.
74.36% churn for credit card users with 3 calls (lowest among high-call users).
Higher churn for direct debit & paper check users even with fewer calls.
Insight: Service calls are a major churn driver, especially for non-credit-card users.

Recommendation:
Address complaints effectively within 1-2 calls to prevent further escalation.
Introduce automated issue resolution (self-service portals, AI chatbots) for common concerns.

**11. Churn Rate for Multi-Call Customers by Contract Type**

Key Finding:
Month-to-Month customers with 2+ calls: 82.30% churn.
One-Year customers with 2+ calls: 40.70% churn.
Two-Year customers with 2+ calls: 11.62% churn.
Insight: Month-to-month customers with repeated service issues are almost guaranteed to churn.

Recommendation:
Offer proactive service interventions to month-to-month customers with multiple calls.
Encourage at-risk customers to switch to long-term contracts with better support benefits.

**12. Churn Rate Comparison: Less Than 2 Calls vs. More Than 2 Calls**
Key Finding:
Less than 2 calls: Lower churn.
More than 2 calls: Churn spikes dramatically, especially for month-to-month customers.
Insight: Multiple calls are a key churn predictor.

Recommendation:
Address customer complaints efficiently within one or two calls.
Use machine learning to predict and flag high-risk customers for retention efforts.

**Overall Business Recommendations:**

- Improve First-Call Resolution: Reduce repeat service calls by training agents and enhancing issue resolution strategies.
- Encourage Long-Term Contracts: Offer discounts for switching from month-to-month to annual plans.
- Promote Automated Payments: Incentivize paper check users to switch to credit cards or direct debits.
- Prioritize High-Risk Customers: Proactively engage customers who have made 2+ service calls with special retention offers.
- Enhance Self-Service Options: Deploy AI-driven chatbots and self-help portals to reduce call volume.
- Optimize Support for High-Value Customers: Provide dedicated support teams for premium users with frequent service needs.
- Implement Predictive Churn Analytics: Use machine learning to flag customers at risk of churn due to multiple service interactions.

## Churn Reason & Category Analysis

We investigate why customers leave and categorize churn causes.

- "Churn Label"
- "Churn Category"
- "Churn Reason"
- "Churned" 

<span style="font-size: 20px;">Question: What are the top 3 most common Churn Reasons within each Churn Category, and how does the churn rate for these reasons compare?

In [32]:
run_sql("""
    WITH churn_reason_category AS (
        SELECT 
            "Churn Category", 
            "Churn Reason", 
            COUNT(*) AS total_reason,
            SUM("Churned") AS churned
        FROM customer_churn_table
        GROUP BY "Churn Category", "Churn Reason"
    )
    SELECT 
        "Churn Category", 
        "Churn Reason", 
        ROUND(SUM(churned) * 100.0 / SUM(total_reason), 2) AS churn_rate
    FROM churn_reason_category
    GROUP BY "Churn Category", "Churn Reason"
    ORDER BY "Churn Category", churn_rate DESC
    LIMIT 3;
""")

,Churn Category,Churn Reason,churn_rate
0,Attitude,Attitude of service provider,None
1,Attitude,Attitude of support person,None
2,Competitor,Competitor had better devices,None


<span style="font-size: 20px;">Question: What is the distribution of Churned vs Non-Churned customers for each Payment Method, and how does this affect the churn rate?

In [33]:
run_sql("""
WITH churn_distribution AS (
    SELECT 
        "Payment Method", 
        COUNT(*) AS total_customers,
        SUM(CASE WHEN "Churn Label" = 'Yes' THEN 1 ELSE 0 END) AS churned,
        SUM(CASE WHEN "Churn Label" = 'No' THEN 1 ELSE 0 END) AS not_churned
    FROM customer_churn_table
    GROUP BY "Payment Method"
)
SELECT 
    "Payment Method",
    churned, 
    not_churned, 
    ROUND(churned * 100.0 / total_customers, 2) AS churn_rate
FROM churn_distribution
ORDER BY churn_rate DESC;
""")

,Payment Method,churned,not_churned,churn_rate
0,Paper Check,140,230,37.84
1,Direct Debit,1262,2425,34.23
2,Credit Card,367,2236,14.10


<span style="font-size: 20px;">Question: How frequently does each Churn Reason appear within its corresponding Churn Category, and which categories have the most diverse set of churn reasons

In [34]:
run_sql("""
WITH reason_count AS (
    SELECT 
        "Churn Category",
        "Churn Reason",
        COUNT(*) AS reason_occurrences
    FROM customer_churn_table
    GROUP BY "Churn Category", "Churn Reason"
),
category_diversity AS (
    SELECT 
        "Churn Category",
        COUNT(DISTINCT "Churn Reason") AS unique_reasons_count
    FROM customer_churn_table
    GROUP BY "Churn Category"
)
SELECT 
    rc."Churn Category",
    rc."Churn Reason",
    rc.reason_occurrences,
    cd.unique_reasons_count AS total_reasons_in_category
FROM reason_count rc
JOIN category_diversity cd ON rc."Churn Category" = cd."Churn Category"
ORDER BY cd.unique_reasons_count DESC, rc.reason_occurrences DESC;
""")

,Churn Category,Churn Reason,reason_occurrences,total_reasons_in_category
0,Dissatisfaction,Product dissatisfaction,73,7
1,Dissatisfaction,Network reliability,69,7
2,Dissatisfaction,Service dissatisfaction,60,7
3,Dissatisfaction,Limited range of services,35,7
4,Dissatisfaction,Lack of self-service on Website,26,7
5,Dissatisfaction,Poor expertise of online support,12,7
6,Dissatisfaction,Poor expertise of phone support,11,7
7,Competitor,Competitor made better offer,303,4
8,Competitor,Competitor had better devices,297,4
9,Other,Don't know,123,4


### Key Insights

1. Top Churn Reasons by Category:
Attitude: Both "Attitude of support person" and "Attitude of service provider" have the highest churn rates of 100%. This suggests that poor customer service, either from support or service providers, is a critical factor in churn.
Competitor: "Competitor had better devices" is a major churn reason, with a 100% churn rate, highlighting the competitive nature of the market.

2. Churn Rate by Payment Method:
Customers paying via Paper Check have the highest churn rate (37.84%), followed by Direct Debit (34.23%) and Credit Card (14.10%).

3. Churn Reason Frequency:
The Dissatisfaction category has the highest diversity with 7 unique churn reasons, including "Product dissatisfaction" and "Network reliability," indicating multiple pain points for customers.
The Competitor category is also diverse, but with fewer churn reasons (4), like "Better devices" and "More data."

**Findings:**

- Customer Service and Attitude: Poor customer service, particularly related to the attitude of support staff and service providers, is a significant churn driver.
- Competition: Customers leave for competitors offering better products or services, particularly in areas like device quality and data packages.
- Payment Method and Churn: Customers using Paper Check have a higher churn rate, possibly due to inconveniences associated with the payment method.
- Dissatisfaction as a Broad Driver: Dissatisfaction due to service quality and product issues is a common churn reason, suggesting a need for service improvements.

**Overall Business Recommendations:**

- Improve Customer Service: Focus on enhancing the quality of customer support and service provider interactions, potentially through training and better response times, to reduce churn in the Attitude category.
- Competitive Analysis: Strengthen product offerings by improving devices and expanding data plans to stay competitive, especially for customers in the Competitor category.
- Payment Method Optimization: Encourage customers to switch from Paper Check to more convenient methods like Direct Debit or Credit Card, which have lower churn rates.
- Address Customer Dissatisfaction: Focus on resolving issues related to service quality, network reliability, and product offerings to reduce churn in the Dissatisfaction category.
- Targeted Retention Strategies: Develop specific retention campaigns for high-churn groups, like those using Paper Check or facing dissatisfaction with the service, offering incentives or better service plans.

# Recommendations & Strategic Actions

**1. Customer Support Interaction Analysis**

Churn increases significantly with customer service calls:
Customers with 3+ service calls have a churn rate above 87%, reaching 100% at 5+ calls.
Churned customers make 2.43 calls on average, whereas non-churned customers make only 0.37 calls.

Recommendation:
    Implement a proactive customer service escalation process: Identify customers making more than 2 calls and route them to a higher-tier support team to resolve issues effectively.
    Monitor sentiment in support interactions to preemptively identify dissatisfaction trends.

**2. Contract Type Analysis**

Month-to-Month contracts have the highest churn rate (46.12%), compared to One-Year (10.69%) and Two-Year (2.46%) contracts.
Customers with longer-term contracts are significantly more loyal.

Recommendation: 
    Incentivize longer-term contracts through discounts, exclusive perks, or bundled offers. Identify high-churn-risk month-to-month customers and offer personalized retention deals before they decide to leave.

**3. Payment Method & Churn**

Paper Check and Direct Debit customers have significantly higher churn rates (37.84% & 34.23%) compared to Credit Card users (14.10%).
Credit Card users tend to have more stable, long-term subscriptions.

Recommendation:     
    Encourage customers to switch to automated payments (Credit Card or Direct Debit) through discounts or reward programs. Identify Paper Check users and offer incentives to transition to digital billing.

**4. Payment Method & Contract Type Impact**

Month-to-Month Paper Check users have the highest churn rate (57.14%), while Two-Year Credit Card users have the lowest (1.56%).
Long-term contract + Credit Card = Low churn.

Recommendation:
    Target Paper Check users with retention campaigns: Personalized outreach offering auto-payment discounts and long-term contract benefits. Create bundled incentives (e.g., loyalty rewards for Credit Card payments and contract extensions).

**5. High-Value Customers & Churn**

Among high-value customers (Monthly Charge > $50 & at least 1 customer service call):
Paper Check & Direct Debit customers churn at over 70%.
Credit Card customers churn less (46.75%) but are still at risk.

Recommendation: 
    For high-value customers, prioritize premium customer service channels (e.g., dedicated account managers or priority support).
    Offer personalized loyalty rewards for those opting for Credit Card payments.

**6. Churn Reasons & Categories**

Most Common Churn Reasons
"Attitude of Support Person" and "Competitor Offers" have 100% churn rates.
Dissatisfaction, Competitor, and Price are the top churn categories.
Diversity of Churn Reasons
Dissatisfaction has the most diverse churn reasons (7), indicating multiple service pain points.
Competitor-related churn is driven by better offers, devices, and higher speeds.
Price concerns are a significant factor but have fewer reasons.

Recommendation:
    Train customer support staff extensively on empathy, problem resolution, and customer retention strategies.
    Offer a competitive pricing & service structure:
    Benchmark against competitor pricing and data speeds.
    Provide limited-time loyalty discounts to counter competitor promotions.

**7. Churn Rate by Customer Service Calls & Payment Methods**

All payment methods show a 100% churn rate at 4-5 service calls, confirming that unresolved customer issues directly lead to churn.
Paper Check users are at the highest risk of dissatisfaction and churn.

Recommendation:
    Customer sentiment tracking: Implement real-time churn risk alerts based on service call volume & sentiment analysis.
    Proactive customer support: Contact customers before they reach 3+ calls and resolve pain points faster.

# Conclusion

Our analysis reveals critical factors driving customer churn, highlighting issues in service usage, plan structures, customer support interactions, and churn motivations. Customers with shorter account tenures show lower service usage but still face significant charges, suggesting a potential mismatch between pricing and perceived value. Those with international plans churn slightly less, but high international usage leads to substantial extra charges, which could be a source of dissatisfaction. Moreover, customers on unlimited data plans consume far more but are not necessarily retained better, indicating that value perception, rather than sheer data limits, influences retention. High-churn states may require localized strategies, while low local call usage correlates with high churn, suggesting that less-engaged customers are at risk.

Customer support interactions strongly impact churn, with a drastic rise in churn for those making multiple service calls. Month-to-month contracts, paper check payments, and direct debit users exhibit the highest churn, especially when combined with multiple service calls, signaling frustration with billing and service experiences. Furthermore, churn reasons reveal that dissatisfaction, competitor offers, and pricing are dominant drivers. Attitude-related churn reaches 100%, indicating that negative service interactions are deal-breakers. Dissatisfaction with product offerings, network reliability, and customer service expertise also contribute significantly, showing gaps in perceived value and support.

To combat churn, businesses should enhance customer engagement early, improving perceived value for new customers while introducing targeted retention offers for at-risk segments. Addressing high extra international charges, optimizing unlimited data pricing, and improving customer service—especially for high-contact users—are crucial. Offering proactive support and ensuring contract flexibility can reduce churn, while payment incentives can shift customers away from high-risk payment methods. Lastly, deeper analysis into state-wise churn patterns and competitor offerings can help refine localized strategies, ultimately improving customer satisfaction and retention.

**Final Business Actions:**

- Improve Customer Support Quality: Train staff, monitor sentiment, and offer proactive escalations.
- Encourage Longer-Term Contracts: Offer discounts and rewards for One-Year and Two-Year plans.
- Promote Digital Payments: Incentivize Credit Card usage to reduce churn risk.
- Personalize Retention Strategies: Identify high-risk customers based on service calls, contract type, and payment method, and offer  targeted interventions.
- Monitor Competitor Offerings & Price Sensitivity: Adjust pricing and service features to remain competitive.

**Closing Connection**

In [35]:
#Closing Connection 
cursor.close()
conn.close()